In [37]:
import sqlite3
conn = sqlite3.connect("/Users/uzair/fractal/dataset/data.db")
curr = conn.cursor()

In [38]:
curr.execute("SELECT * FROM train")
train_data = curr.fetchall()

In [39]:
curr.execute("SELECT * FROM test")
test_data = curr.fetchall()

In [40]:
header = ["ID","Customer_Type","Customer_Age",
          "Source_location_type","Destination_location_Type","Distance","Peak","Traffic","Time",
          "Weather","Is_weekend","Customer_demand","Price"
          ]

header_test = ["ID","Customer_Type","Customer_Age",
          "Source_location_type","Destination_location_Type","Distance","Peak","Traffic","Time",
          "Weather","Is_weekend","Customer_demand"
          ]

In [41]:
import csv
with open("train.csv","w", newline="") as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(header)
    csvwriter.writerows(train_data)

In [42]:
with open("test.csv","w", newline="") as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(header_test)
    csvwriter.writerows(test_data)

In [43]:
import pandas as pd
df_train = pd.read_csv("train.csv")
df_val =pd.read_csv("test.csv")

In [44]:
df_train.shape

(57202, 13)

In [45]:
df_val.shape

(12472, 12)

In [46]:
df_train.columns

Index(['ID', 'Customer_Type', 'Customer_Age', 'Source_location_type',
       'Destination_location_Type', 'Distance', 'Peak', 'Traffic', 'Time',
       'Weather', 'Is_weekend', 'Customer_demand', 'Price'],
      dtype='object')

In [47]:
df_val.columns

Index(['ID', 'Customer_Type', 'Customer_Age', 'Source_location_type',
       'Destination_location_Type', 'Distance', 'Peak', 'Traffic', 'Time',
       'Weather', 'Is_weekend', 'Customer_demand'],
      dtype='object')

In [48]:
df_train.set_index("ID",inplace=True)


In [49]:
df_train.head(5)

,Customer_Type,Customer_Age,Source_location_type,Destination_location_Type,Distance,Peak,Traffic,Time,Weather,Is_weekend,Customer_demand,Price
ID,,,,,,,,,,,,
0x2cd1,New,14.0,Inside_City,Outside_City,7.6,No,48.6,Morning,Sunny,No,7.36,6.84
0x10985,New,25.0,Outside_City,Inside_City,8.1,Yes,85.8,Midday,Sunny,No,6.42,8.71
0xae8d,New,19.0,Inside_City,Inside_City,13.1,No,86.0,Morning,Sunny,Yes,NaN,10.16
0x86ab,Regular,46.0,Inside_City,Inside_City,4.7,No,67.1,Midnight,Windy,No,6.19,13.44
0x8c,Inactive,52.0,Inside_City,Outside_City,2.6,No,39.8,Midday,Rainy,No,5.83,11.49


In [50]:
df_val.head(5)

,ID,Customer_Type,Customer_Age,Source_location_type,Destination_location_Type,Distance,Peak,Traffic,Time,Weather,Is_weekend,Customer_demand
0,0x108fd,Passive,26.0,Inside_City,Outside_City,6.7,No,63.8,Night,Windy,No,5.81
1,0xaa0f,Passive,30.0,Inside_City,Outside_City,3.0,Yes,84.6,Evening,Windy,No,6.74
2,0x791d,Passive,24.0,Inside_City,Outside_City,6.8,No,63.6,Morning,Sunny,No,4.77
3,0x149c,New,20.0,Inside_City,Outside_City,6.0,No,####,Night,Rainy,No,1.97
4,0x1a77,Inactive,59.0,Inside_City,Inside_City,4.5,Yes,69.8,Morning,Sunny,No,9.68


In [51]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.model_selection import train_test_split

In [52]:
df_train.drop_duplicates(inplace=True)
df_val.drop_duplicates(inplace=True)
df_train.replace("####", np.nan,inplace=True)
df_val.replace("####", np.nan,inplace=True)

In [53]:
X = df_train.drop("Price",axis=1)
y = df_train['Price']

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [55]:
numerical_cols = ["Customer_Age","Distance","Traffic","Customer_demand"]

In [56]:
categorical_cols = ["Customer_Type","Source_location_type","Destination_location_Type",
                    "Peak","Time","Weather","Is_weekend"]

In [57]:
numerical_xfer =Pipeline(steps=[("imputer",SimpleImputer(strategy='mean')),("scaler",StandardScaler())])

In [58]:
cat_xfer = Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy="most_frequent")),
        ("onehot",OneHotEncoder(
            handle_unknown="ignore"
        ))
    ]
)

In [59]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num",numerical_xfer,numerical_cols),
        ("cat",cat_xfer,categorical_cols)
    ]
)

In [61]:
lr_pipeline = Pipeline(
    steps=[
        ("preprocessor",preprocessor),("regressor",LinearRegression(
        ))
    ]
)

In [62]:
lasso_pipeline = Pipeline(
    steps=[
        ('preprocessor',preprocessor),
        ("regressor",Lasso(alpha=0.1))
    ]
)

In [63]:
lr_pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Customer_Age', 'Distance',
                                                   'Traffic',
                                                   'Customer_demand']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Customer_Type',
                                                   'Source_location_type',
                                                   'Destination_location_Type',
                                                   'Peak', 'Time', 'Weather',
                                                   'Is_weekend'])])),
                ('regressor', LinearRegression())])

In [64]:
lasso_pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Customer_Age', 'Distance',
                                                   'Traffic',
                                                   'Customer_demand']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Customer_Type',
                                                   'Source_location_type',
                                                   'Destination_location_Type',
                                                   'Peak', 'Time', 'Weather',
                                                   'Is_weekend'])])),
                ('regressor', Lasso(alpha=0.1))])

In [66]:
lr_pred = lr_pipeline.predict(X_test)
lasso_pred = lasso_pipeline.predict(X_test)

In [67]:
def evalution(actual,preds):
    r2 = r2_score(actual,preds)
    return max(0,100*r2)

In [68]:
lr_score = evalution(y_test,lr_pred)

In [69]:
lr_score

55.24604036462766

In [70]:
lasso_score = evalution(y_test,lasso_pred)

In [71]:
lasso_score

50.85931788105883

In [72]:
y_pred = lasso_pipeline.predict(df_val)

In [73]:
y_pred

array([12.04040855, 15.33478146,  8.47676906, ..., 10.43732383,
       12.33918095, 11.64245428])

In [74]:
prediction_df = pd.DataFrame({"ID":df_val['ID'],"Price":y_pred})

In [75]:
prediction_df.head()

,ID,Price
0,0x108fd,12.040409
1,0xaa0f,15.334781
2,0x791d,8.476769
3,0x149c,8.222088
4,0x1a77,12.851271


In [76]:
prediction_df.to_csv("prediction.csv",index=False)